In [1]:

import numpy as np
import pandas as pd

matrix = np.array([
    [0.081, 0.075, 0.105, 0.101],
    [0.058, 0.073, 0.087, 0.093],
    [0.052, 0.087, 0.073, 0.09],
    [0.039, 0.079, 0.081, 0.08],
    [0.080, 0.077, 0.239, 0.071],
    [0.278, 0.058, 0.117, 0.062],
    [0.038, 0.123, 0.015, 0.131],
    [0.028, 0.089, 0.018, 0.134],
    [0.036, 0.119, 0.108, 0.095],
    [0.098, 0.093, 0.121, 0.081],
    [0.013, 0.127, 0.038, 0.063]
])

criteria = ['C1', 'C2', 'C3', 'C4']
sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8', 'Site9', 'Site10', 'Site11']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.16376103, 0.29605439, 0.46393757, 0.076247])

# V-shape preference thresholds (can be tuned)
p_list = [2.998, 1.441, 0.783, 3.957]

def normalize_matrix(matrix, criteria_types):
    norm_matrix = np.zeros_like(matrix, dtype=float)
    for j in range(matrix.shape[1]):
        col = matrix[:, j]
        if criteria_types[j] == 'max':
            norm_matrix[:, j] = (col - np.min(col)) / (np.max(col) - np.min(col))
        else:  # 'min'
            norm_matrix[:, j] = (np.max(col) - col) / (np.max(col) - np.min(col))
    return norm_matrix

def vshape_preference(d, p):
    return np.clip(d / p, 0, 1)

def calculate_preference_matrix(norm_matrix, criteria_types, weights, p_list):
    n = norm_matrix.shape[0]
    m = norm_matrix.shape[1]
    preference_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            pref_sum = 0
            for k in range(m):
                if criteria_types[k] == 'max':
                    diff = norm_matrix[i, k] - norm_matrix[j, k]
                else:  # 'min'
                    diff = norm_matrix[j, k] - norm_matrix[i, k]
                pref = vshape_preference(diff, p_list[k])
                pref_sum += weights[k] * pref
            preference_matrix[i, j] = pref_sum
    return preference_matrix

def calculate_flows(preference_matrix):
    n = preference_matrix.shape[0]
    phi_plus = preference_matrix.sum(axis=1) / (n - 1)
    phi_minus = preference_matrix.sum(axis=0) / (n - 1)
    net_flows = phi_plus - phi_minus
    return phi_plus, phi_minus, net_flows

def promethee_ii(matrix, criteria_types, weights, p_list, sites, criteria):
    norm_matrix = normalize_matrix(matrix, criteria_types)
    preference_matrix = calculate_preference_matrix(norm_matrix, criteria_types, weights, p_list)
    phi_plus, phi_minus, net_flows = calculate_flows(preference_matrix)
    
    # Prepare results DataFrame
    df = pd.DataFrame(matrix, columns=criteria, index=sites)
    df['Leaving Flow'] = phi_plus
    df['Entering Flow'] = phi_minus
    df['Net Flow'] = net_flows
    df['Rank'] = df['Net Flow'].rank(ascending=False, method='min').astype(int)
    df = df.sort_values('Rank')
    return df

# Run PROMETHEE II
result_df = promethee_ii(matrix, criteria_types, weights, p_list, sites, criteria)
print(result_df)


           C1     C2     C3     C4  Leaving Flow  Entering Flow  Net Flow  \
Site5   0.080  0.077  0.239  0.071      0.412670       0.063816  0.348854   
Site9   0.036  0.119  0.108  0.095      0.189234       0.055199  0.134035   
Site10  0.098  0.093  0.121  0.081      0.162589       0.065949  0.096640   
Site6   0.278  0.058  0.117  0.062      0.155238       0.149653  0.005585   
Site1   0.081  0.075  0.105  0.101      0.100669       0.107506 -0.006837   
Site11  0.013  0.127  0.038  0.063      0.129609       0.180910 -0.051300   
Site3   0.052  0.087  0.073  0.090      0.065558       0.136015 -0.070457   
Site2   0.058  0.073  0.087  0.093      0.065912       0.139244 -0.073332   
Site4   0.039  0.079  0.081  0.080      0.059461       0.138734 -0.079273   
Site7   0.038  0.123  0.015  0.131      0.118917       0.218470 -0.099553   
Site8   0.028  0.089  0.018  0.134      0.039070       0.243433 -0.204362   

        Rank  
Site5      1  
Site9      2  
Site10     3  
Site6      4  
